<a href="https://colab.research.google.com/github/WaleedAhmed565/Ai-lab1/blob/main/pathfinding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import heapq

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_algorithm(grid, start, end):
    open_list = []
    closed_list = set()

    start_node = Node(start)
    end_node = Node(end)
    heapq.heappush(open_list, start_node)

    while open_list:
        current_node = heapq.heappop(open_list)
        if current_node.position == end_node.position:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]

        closed_list.add(current_node.position)
        for new_position in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Check if the node position is within grid bounds and is not an obstacle
            if (0 <= node_position[0] < len(grid)) and (0 <= node_position[1] < len(grid[0])) and grid[node_position[0]][node_position[1]] == 0:
                if node_position in closed_list:
                    continue

                neighbor_node = Node(node_position, current_node)
                neighbor_node.g = current_node.g + 1
                neighbor_node.h = heuristic(neighbor_node.position, end_node.position)
                neighbor_node.f = neighbor_node.g + neighbor_node.h
                if any(neighbor_node == open_node and neighbor_node.g >= open_node.g for open_node in open_list):
                    continue
                heapq.heappush(open_list, neighbor_node)

    return None

def print_path(grid, path):
    for position in path:
        grid[position[0]][position[1]] = 'P'
    for row in grid:
        print(" ".join(str(x) for x in row))
grid = [
    [0, 0, 1, 0, 0],
    [0, 1, 1, 0, 1],
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

start = (0, 0)
end = (4, 4)

path = a_star_algorithm(grid, start, end)
if path:
    print_path(grid, path)
else:
    print("No path found")



P 0 1 0 0
P 1 1 0 1
P P P P P
0 1 1 1 P
0 0 0 0 P


In [13]:
import heapq

def astar_water_jug(start, goal):
    open_list = []
    heapq.heappush(open_list, (0, start))

    came_from = {}
    g_score = {start: 0}

    def heuristic(state):
        return abs(state[0] - goal) + abs(state[1] - goal)

    def get_neighbors(state):
        x, y = state
        neighbors = []
        jug1_capacity = 4
        jug2_capacity = 3


        neighbors.append((jug1_capacity, y))
        neighbors.append((x, jug2_capacity))
        neighbors.append((0, y))
        neighbors.append((x, 0))
        transfer = min(x, jug2_capacity - y)
        neighbors.append((x - transfer, y + transfer))
        transfer = min(y, jug1_capacity - x)
        neighbors.append((x + transfer, y - transfer))

        return neighbors

    while open_list:
        _, current = heapq.heappop(open_list)

        if current[0] == goal or current[1] == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return path

        for neighbor in get_neighbors(current):
            tentative_g_score = g_score[current] + 1
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score + heuristic(neighbor)
                heapq.heappush(open_list, (f_score, neighbor))

    return None


start = (0, 0)
goal = 2
path = astar_water_jug(start, goal)

if path:
    for state in path:
        print(f"Jug1: {state[0]} liters, Jug2: {state[1]} liters")

Jug1: 0 liters, Jug2: 3 liters
Jug1: 3 liters, Jug2: 0 liters
Jug1: 3 liters, Jug2: 3 liters
Jug1: 4 liters, Jug2: 2 liters


In [2]:
import random

def calculate_conflicts(board):
    conflicts = 0
    n = len(board)

    for i in range(n):
        for j in range(i + 1, n):
            if board[i] == board[j] or abs(board[i] - board[j]) == j - i:
                conflicts += 1
    return conflicts

def get_best_move(board, col):
    n = len(board)
    min_conflicts = calculate_conflicts(board)
    best_row = board[col]

    for row in range(n):
        if board[col] == row:
            continue
        new_board = board[:]
        new_board[col] = row
        conflicts = calculate_conflicts(new_board)
        if conflicts < min_conflicts:
            min_conflicts = conflicts
            best_row = row

    return best_row, min_conflicts

def hill_climbing_8_queen():
    n = 8
    board = [random.randint(0, n-1) for _ in range(n)]

    while True:
        current_conflicts = calculate_conflicts(board)
        if current_conflicts == 0:
            return board

        moves_made = False
        for col in range(n):
            best_row, best_conflicts = get_best_move(board, col)
            if best_conflicts < current_conflicts:
                board[col] = best_row
                current_conflicts = best_conflicts
                moves_made = True

        if not moves_made:
            break

    return board

solution = hill_climbing_8_queen()

def print_board(board):
    n = len(board)
    for row in range(n):
        line = ""
        for col in range(n):
            if board[col] == row:
                line += "Q "
            else:
                line += ". "
        print(line)

if calculate_conflicts(solution) == 0:
    print("Solution found!")
    print_board(solution)
else:
    print("Local optimum reached. Board:")
    print_board(solution)

Local optimum reached. Board:
. . . Q Q . . . 
. . . . . . Q . 
Q . . . . . . . 
. . Q . . . . . 
. . . . . Q . . 
. Q . . . . . . 
. . . . . . . . 
. . . . . . . Q 


In [ ]:
import math

def print_board(board):
    for row in board:
        print('| ' + ' | '.join(row) + ' |')
    print()

def check_winner(board, player):
    for row in range(3):
        if board[row][0] == board[row][1] == board[row][2] == player:
            return True
    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] == player:
            return True
    if board[0][0] == board[1][1] == board[2][2] == player:
        return True
    if board[0][2] == board[1][1] == board[2][0] == player:
        return True
    return False

def is_board_full(board):
    for row in board:
        if ' ' in row:
            return False
    return True

def minimax(board, depth, is_maximizing):
    if check_winner(board, 'O'):
        return 1
    if check_winner(board, 'X'):
        return -1
    if is_board_full(board):
        return 0

    if is_maximizing:
        best_score = -math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == ' ':
                    board[i][j] = 'O'
                    score = minimax(board, depth + 1, False)
                    board[i][j] = ' '
                    best_score = max(best_score, score)
        return best_score
    else:
        best_score = math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == ' ':
                    board[i][j] = 'X'
                    score = minimax(board, depth + 1, True)
                    board[i][j] = ' '
                    best_score = min(best_score, score)
        return best_score

def best_move(board):
    best_score = -math.inf
    move = None
    for i in range(3):
        for j in range(3):
            if board[i][j] == ' ':
                board[i][j] = 'O'
                score = minimax(board, 0, False)
                board[i][j] = ' '
                if score > best_score:
                    best_score = score
                    move = (i, j)
    return move

def play_game():
    board = [[' ' for _ in range(3)] for _ in range(3)]
    print("Welcome to Tic-Tac-Toe! You are 'X' and AI is 'O'.")
    print_board(board)

    while True:
        human_move = input("Enter your move (row and column number, 0-2) separated by a space: ")
        row, col = map(int, human_move.split())

        if board[row][col] != ' ':
            print("Invalid move. Try again.")
            continue

        board[row][col] = 'X'
        print_board(board)

        if check_winner(board, 'X'):
            print("Congratulations! You win!")
            break
        if is_board_full(board):
            print("It's a draw!")
            break

        print("AI is making its move...")
        ai_move = best_move(board)
        board[ai_move[0]][ai_move[1]] = 'O'
        print_board(board)

        if check_winner(board, 'O'):
            print("AI wins! Better luck next time.")
            break
        if is_board_full(board):
            print("It's a draw!")
            break

play_game()



Welcome to Tic-Tac-Toe! You are 'X' and AI is 'O'.
|   |   |   |
|   |   |   |
|   |   |   |

